### Oliver Mazariegos - 16043

# Redes Neuronales
## Clasificación de tipos de ropa con imagenes


En este notebook se estará creando una red neuronal capaz de clasificar ropa a partir de una foto. Las fotos fueron obtenidas de un dataset publicado en kaggle https://www.kaggle.com/zalando-research/fashionmnist y fue descargado directo de su repositorio original en github https://github.com/zalandoresearch/fashion-mnist.

## Librerias utilizadas

In [10]:
import numpy as np
import mnist_reader
from functools import reduce
from scipy import optimize as op
from sklearn.model_selection import train_test_split

## Funciones utiles para el manejo de matrices

In [11]:
flatten_list_of_arrays = lambda list_of_arrays: reduce (
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()],dtype=np.float64),
    list_of_arrays
)

def flat_matrixes(lista):
    flatten = []
    for matris in lista:
        # se aplanan por dentro
        flatten = [*flatten,*(matris.flatten())]
    # Se terminan de aplanar
    return np.array(flatten).flatten()
    


def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)
    for i in range(layers - 1):
        steps[i+1] = steps[i] + sizes[i]
    return [
        flat_thetas[steps[i]:steps[i+1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

Las funciones anteriores son aplanadoras e infladoras de matrices. Esto debido que utilizaremos matrices de transición, peso y error para hacer calculos para las capas y neuronas dentro de la red neuronal. Pero para el algoritmo de optimización este recibe y devuelve estas matrices de forma aplanada en una lista.

## Funcion de pertenencia

In [12]:
def sigmoid(x):
    return 1.0/(1 + np.exp(-x))

La funcion que utilizaremos para calcular la probabilidad de pertenencia sera la sigmoide. Esta funcion es utilizada para muchas curvas de aprendizaje de sistemas complejos.

## Feed Forward

Una parte fundamental del algoritmo que emplearemos para la creación de la red neuronal es el `Feed Forward`. Este es un algoritmo que calcula las matrices de activacion para cada una de las capas de la red neuronal. Siendo la primer capa las entradas de la red (la imagen) y la ultima capa el resultado que genera (la prediccion).

In [13]:
def feed_forward(thetas, X):
    # Lista de activaciones
    a = [X] 
    for i in range(len(thetas)): 
        a.append(
            # a
            sigmoid(
                # z
                np.matmul(
                    # agrega bias
                    np.hstack((
                        np.ones(len(a[i])).reshape(len(a[i]), 1),
                        a[i]
                    )),
                    thetas[i].T
                )
            )
        )
    return a 

## Back_propagation

El `Back Propagation` es el algoritmo encargado de detectar el error y el impacto de cada capa sobre el error del resultado final. El proposito de este algoritmo es calcular las matrices Delta el cual contiene el gradiente; que es utilizado para optimizar el modelo.

In [14]:
def back_propagation(flat_thetas, shapes, X, Y):
    # 1
    delta = []
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
     # 2.2
    a = feed_forward(thetas, X)
    deltas = [ *range(layers -1), a[-1] - Y ] 
    # 2.4
    for i in range(layers-2, 0, -1): # loop desde la ultima capa hasta la segunda (en reversa)
        deltas[i] = np.matmul(deltas[i+1],(thetas[i])[:, 1:]) * (a[i] * (1 - a[i]))
    # 2.5 y 3 
    for i in range(layers-1): # loop de capa 0 a capa L-1
        delta.append((np.matmul(
            deltas[i+1].T, 
            np.hstack(( # se agrega bias a 
                        np.ones(len(a[i])).reshape(len(a[i]), 1),
                        a[i]
                    ))
        )) / m)
    return flat_matrixes(delta)

## Función de Costo

La función de costo es la encargada de detectar que tan preciso o impreciso es la prediccion de nuestra red neuronal.

In [15]:
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

## Dataset

El dataset esta conformado por imagenes de 28x28 resultando en 784 bits. En cada bit hay un valor entre 0 y 255, donde 0 representa al color negro y 255 al color blanco. Ademas el dataset incluye a que tipo de prenda pertenece la imagen. Existen 10 categorias de ropa representadas por un numero de la siguiente manera:

* 0 T-shirt/top
* 1 Trouser
* 2 Pullover
* 3 Dress
* 4 Coat
* 5 Sandal
* 6 Shirt
* 7 Sneaker
* 8 Bag
* 9 Ankle boot



## Train Set

El dataset de entrenamiento contiene 60000 filas, cada fila representando una prenda distinta. Debido a que la derivada de la sigmoide esta acotada normalizaremos el dataset diviendolo dentro de 1000 para que la derivada no se indefina.

In [16]:
X_train, y_train = mnist_reader.load_mnist('',kind='train')
X = X_train / 1000
m, n = X.shape

Nuestra red neuronal dara como resultado un vector con shape (10,1) mientras que la categoria brindada por el dataset tiene un shape de (1,) por lo que le haremos una conversion a este mismo para que coincidan las shapes a la hora de comparar efectividad.

In [30]:
Y = np.zeros((X.shape[0], 10))
for i in range(m):
    Y[i][y_train[i]] = 1

In [ ]:
Y[0]

## Estructura de la Red Neuronal

La red neuronal tendrá 100 neuronas ocultas y como previamente discutido 10 neuronas de salida.

In [17]:
HIDDEN_NEURONS = 100
OUTPUT_NEURONS = 10

theta_shapes = np.array([
    [HIDDEN_NEURONS, n + 1],
    [OUTPUT_NEURONS, HIDDEN_NEURONS + 1]
])


flat_thetas = flatten_list_of_arrays(
    [np.random.rand(*theta_shape) / 1000 for theta_shape in theta_shapes]
)

## Optimizacion del Modelo

Para optimizar la red neuronal se utilizara la funcion de optimize.minimize la cual recibe un modelo inicial aleatorio para comenzar la optimización, la función de costo, el set de entrenamiento, el resultado y la funcion que genera el gradiente que en nuestro caso es el `Back Propagation`. Ademas recibe como parametro el método de optimización a utilizar que en este caso será `L-BFGS-B`.

In [18]:
result = op.minimize(
    fun=cost_function,
    x0=flat_thetas,
    args=(theta_shapes,X,Y),
    method='L-BFGS-B',
    jac=back_propagation,
    options={'disp':True, 'maxiter':400}
)

## Modelo Resultante

Como discutido anteriormente, la función optimizadora da los tethas optimos en forma de lista, por lo que tendremos que inflarla para obtener su forma matricial y poder utilizarla

In [33]:
result.x

array([ 6.51394812e-01,  6.49216013e-04,  2.65023664e-04, ...,
       -8.46816999e-01, -5.61329764e-01, -3.38209573e+00])

In [34]:
modelo = inflate_matrixes(result.x,theta_shapes)

## Test Set

El dataset de entrenamiento tambien tiene los mismos atributos que el dataset de entrenamiento pero contiene 10000 imagene/prendas que no fueron consideradas en el set de entrenamiento.

In [35]:
X_test, y_test = mnist_reader.load_mnist('', kind='t10k')
X = X_test / 1000
m, n = X.shape
Y = np.zeros((X.shape[0], 10))
for i in range(m):
    Y[i][y_test[i]] = 1

## Prediccion del modelo

In [36]:
prediccion = feed_forward(modelo,X)[-1]

## Certeza del Modelo

Para calcular la certeza del modelo se utilizara la relación de $ \frac{Predicciones correcta}{Número total de prendas} $

In [37]:
def accuracy(prediction, real):
    correctos = 0
    for i in range(len(real)):
        if np.argmax(real[i]) == np.argmax(prediction[i]):
            correctos += 1
    return correctos/len(real)
        

In [38]:
print("La certeza de la red neuronal es de: ",accuracy(prediccion,Y))

La certeza de la red neuronal es de:  0.8822


## Conclusión

Se logró obtener una red neuronal que clasificá ropa con tan solo 400 iteraciones de aprendizaje con el algoritmo de `Back Propagation con Feed Forward`, utilizando 100 neuronas ocultas con una precisión del 88%